In [29]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
# Define the URL of the Google Sheets CSV export link
sheet_url = "https://docs.google.com/spreadsheets/d/1ioUjhnz6ywSpEbARI-G3RoPyO0NRBqrJnWf-7C_eirs/export?format=csv&id=1ioUjhnz6ywSpEbARI-G3RoPyO0NRBqrJnWf-7C_eirs&gid=1854892322"

# Read the data from the Google Sheets URL into a DataFrame
df = pd.read_csv(sheet_url)
print('The original dataset contains', df.shape[0], 'rows and', df.shape[1], 'columns.')

The original dataset contains 17161 rows and 20 columns.


In [31]:
# Display the DataFrame
print(df.head())

            Timestamp How old are you?  \
0  4/11/2023 11:02:00            35-44   
1  4/11/2023 11:02:07            25-34   
2  4/11/2023 11:02:12            35-44   
3  4/11/2023 11:02:15            25-34   
4  4/11/2023 11:02:25            18-24   

                                   Industry  \
0        Government & Public Administration   
1  Galleries, Libraries, Archives & Museums   
2              Education (Higher Education)   
3              Education (Higher Education)   
4             Accounting, Banking & Finance   

                     Functional area of job                  Job title  \
0              Engineering or Manufacturing         Materials Engineer   
1  Galleries, Libraries, Archives & Museums   Assistant Branch Manager   
2              Education (Higher Education)  Director of Financial Aid   
3        Government & Public Administration   Administrative Assistant   
4                            Administration        Executive Assistant   

  Job title - addit

## Before analysing any dataset, it needs to be cleaned.
Data Cleaning consists of several essential tasks and techniques, including -
* Handling missing values
* Outlier identification
* Data type conversion
* Standardization and normalization
* Deduplication
* Addressing inconsistent data, formatting, typos and spelling errors

The column names will be renamed to make them more simplified. This is done to make the column names more clean, intuitive and easy to work with during analysis.

In [32]:
for i, col in enumerate(df.columns):
    print(i, col)

0 Timestamp
1 How old are you?
2 Industry
3 Functional area of job
4 Job title
5 Job title - additional context
6 Annual salary (gross)
7 Additional monetary compensation
8 Currency
9 Currency - other
10 Income - additional context
11 Country
12 State
13 City
14 Remote or on-site?
15 Years of experience, overall
16 Years of experience in field
17 Highest level of education completed
18 Gender
19 Race


In [33]:
# Create a dictionary to map old column names to new column names
column_renames = {
    "Timestamp": "submission_date",
    "How old are you?": "age",
    "Industry": "industry",
    "Functional area of job": "job_function",
    "Job title": "job_title",
    "Job title - additional context": "job_title_context",
    "Annual salary (gross)": "annual_salary_gross",
    "Additional monetary compensation": "additional_compensation",
    "Currency": "currency",
    "Currency - other": "currency_other",
    "Income - additional context": "income_context",
    "Country": "country",
    "State": "us_state",
    "City": "city",
    "Remote or on-site?": "remote_or_onsite",
    "Years of experience, overall": "experience_overall",
    "Years of experience in field": "experience_field",
    "Highest level of education completed": "education_level",
    "Gender": "gender",
    "Race": "race"
}

# Rename the columns
df.rename(columns=column_renames, inplace=True)

# Display the renamed DataFrame
print(df.columns)

Index(['submission_date', 'age', 'industry', 'job_function', 'job_title',
       'job_title_context', 'annual_salary_gross', 'additional_compensation',
       'currency', 'currency_other', 'income_context', 'country', 'us_state',
       'city', 'remote_or_onsite', 'experience_overall', 'experience_field',
       'education_level', 'gender', 'race'],
      dtype='object')


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17161 entries, 0 to 17160
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   submission_date          17161 non-null  object 
 1   age                      17161 non-null  object 
 2   industry                 17115 non-null  object 
 3   job_function             17050 non-null  object 
 4   job_title                17161 non-null  object 
 5   job_title_context        3909 non-null   object 
 6   annual_salary_gross      17161 non-null  int64  
 7   additional_compensation  13145 non-null  float64
 8   currency                 17161 non-null  object 
 9   currency_other           90 non-null     object 
 10  income_context           1694 non-null   object 
 11  country                  17161 non-null  object 
 12  us_state                 14177 non-null  object 
 13  city                     17093 non-null  object 
 14  remote_or_onsite      

The 'currency' and 'currency_other' columns can be merged for simplicity.

In [6]:
df['currency'] = np.where(df['currency'] == 'Other', df['currency_other'], df['currency'])
df.drop('currency_other', axis=1, inplace=True)

The entries with null values in the 'industry' column are removed.

In [7]:
df.dropna(subset=['industry'], inplace=True)

For ease of analysis, I will only keep the entries which contain values from the form checkboxes for the 'industry' field.

In [8]:
# List of categories to keep
categories = ['Accounting, Banking & Finance','Agriculture or Forestry','Art & Design',
              'Business or Consulting',
              'Computing or Tech',
              'Education (Primary/Secondary)','Education (Higher Education)','Engineering or Manufacturing','Entertainment',
              'Government and Public Administration',
              'Health care','Hospitality & Events',
              'Insurance',
              'Law','Law Enforcement & Security','Leisure, Sport & Tourism',
              'Marketing, Advertising & PR','Media & Digital',
              'Nonprofits',
              'Property or Construction',
              'Recruitment or HR','Retail',
              'Sales','Social Work',
              'Transport or Logistics',
              'Utilities & Telecommunications']

# Filter DataFrame to only keep rows with category match
df = df[df['industry'].isin(categories)]

The missing values in the 'additional_compensation' column should be replaced with '0'.

In [9]:
# Fill NaN values in 'Extra_Compensation' column with 0
df.fillna({'additional_compensation': 0}, inplace=True)

Entries where 'gender' is not specified will be removed.

In [10]:
# Filter to remove any invalid gender values
valid_genders = ['Man', 'Woman', 'Non-binary']
df = df[df['gender'].isin(valid_genders)]

Entries where 'education_level' is not specified will be removed.

In [11]:
# Drop rows with missing values
df = df.dropna(subset=['education_level'])

From entries where 'race' contains the string 'Another option not listed here or prefer not to answer', that specific string is removed from them.

In [12]:
# Define the pattern to be removed
filter_pattern = re.compile(r'(, )?Another option not listed here or prefer not to answer')

# Replace the pattern with an empty string
df['race'] = df['race'].str.replace(filter_pattern, '', regex=True)

# Drop rows where 'Race' is null or empty
df = df.dropna(subset=['race'])
df.drop(df[df['race'] == ''].index, inplace=True)

In [13]:
df['race'].unique()

array(['White', 'Asian or Asian American, White',
       'Asian or Asian American', 'Hispanic, Latino, or Spanish origin',
       'Hispanic, Latino, or Spanish origin, White',
       'Black or African American, Middle Eastern or Northern African',
       'Black or African American, White', 'Black or African American',
       'Middle Eastern or Northern African',
       'Hispanic, Latino, or Spanish origin, Middle Eastern or Northern African',
       'Native American or Alaska Native, White',
       'Black or African American, Native American or Alaska Native, White',
       'Asian or Asian American, Black or African American',
       'Middle Eastern or Northern African, Native American or Alaska Native, White',
       'Middle Eastern or Northern African, White',
       'Black or African American, Hispanic, Latino, or Spanish origin',
       'Hispanic, Latino, or Spanish origin, Middle Eastern or Northern African, White',
       'Native American or Alaska Native',
       'Asian or Asian

The 'annual_salary_gross' and 'additional_compensation' values will be standardized to GBP.

In [15]:
# Average 2023 rates taken from https://www.ofx.com/en-gb/forex-news/historical-exchange-rates/monthly-average-rates/
# Took avg of AUD & NZD for AUD/NZD

rates = {'USD':0.804368,
         'EUR':0.870293,
         'JPY':0.005738,
         'GBP': 1.0,
         'CHF':0.89535,
         'CAD':0.595765,
         'AUD/NZD':0.514154,
         'ZAR': 0.043632,
         'HKD':0.102741,
         'SEK':0.075851
        }

# Conversion function
def convert_to_gbp(row):
    currency = str(row['currency']).upper()
    if currency in rates:
        row['annual_salary_gross'] *= rates[currency]
        row['additional_compensation'] *= rates[currency]
        row['currency'] = 'GBP'
    return row

# Apply the conversion function to the DataFrame
df = df.apply(convert_to_gbp, axis=1)

Entries with a 'currency' value other than 'GBP' are dropped.

In [16]:
df = df[df['currency'] == 'GBP']

Normalising the 'country' column through the SequenceMatcher imported from the difflib library. A function is created which is used to compare the similarity ratios between each value in the 'country' column and the values in the 'correct_names' array. This 'correct_names' array contains the correct names of all the countries.

Note that this method isn't 100% accurate but it is efficient. With large datasets, it is not possible to go through each entry to normalise the values in a specific column.

In [20]:
from difflib import SequenceMatcher

df['country'] = df['country'].str.replace('.', '', regex=False) # remove periods

#let's specify correct country names
country_names = {"Afghanistan","Albania","Algeria","Andorra","Angola","Anguilla","Antigua & Barbuda","Argentina","Armenia","Aruba","Australia",
                 "Austria","Azerbaijan",
                 "Bahamas","Bahrain","Bangladesh","Barbados","Belarus","Belgium","Belize","Benin","Bermuda","Bhutan","Bolivia",
                 "Bosnia & Herzegovina","Botswana","Brazil","British Virgin Islands","Brunei","Bulgaria","Burkina Faso","Burundi",
                 "Cambodia","Cameroon","Cape Verde","Cayman Islands","Chad","Chile","China","Colombia","Congo","Cook Islands","Costa Rica",
                 "Cote D Ivoire","Croatia","Cuba","Cyprus","Czech Republic",
                 "Denmark","Djibouti","Dominica","Dominican Republic",
                 "England","Ecuador","Egypt","El Salvador","Equatorial Guinea","Estonia","Ethiopia",
                 "Falkland Islands","Faroe Islands","Fiji","Finland","France","French Polynesia","French West Indies",
                 "Gabon","Gambia","Georgia","Germany","Ghana","Gibraltar","Greece","Greenland","Grenada","Guam","Guatemala","Guernsey","Guinea",
                 "Guinea Bissau","Guyana",
                 "Haiti","Honduras","Hong Kong","Hungary",
                 "Iceland","India","Indonesia","Iran","Iraq","Ireland","Isle of Man","Israel","Italy",
                 "Jamaica","Japan","Jersey","Jordan",
                 "Kazakhstan","Kenya","Kuwait","Kyrgyz Republic",
                 "Laos","Latvia","Lebanon","Lesotho","Liberia","Libya","Liechtenstein","Lithuania","Luxembourg",
                 "Macau","Macedonia","Madagascar","Malawi","Malaysia","Maldives","Mali","Malta","Mauritania","Mauritius","Mexico","Moldova",
                 "Monaco","Mongolia","Montenegro","Montserrat","Morocco","Mozambique",
                 "Northen Ireland","Namibia","Nepal","Netherlands","Netherlands Antilles","New Caledonia","New Zealand","Nicaragua","Niger",
                 "Nigeria","Norway",
                 "Oman",
                 "Pakistan","Palestine","Panama","Papua New Guinea","Paraguay","Peru","Philippines","Poland","Portugal","Puerto Rico",
                 "Qatar",
                 "Reunion","Romania","Russia","Rwanda",
                 "Scotland","Saint Pierre & Miquelon","Samoa","San Marino","Satellite","Saudi Arabia","Senegal","Serbia","Seychelles",
                 "Sierra Leone", "Singapore","Slovakia","Slovenia","South Africa","South Korea","Spain","Sri Lanka","St Kitts & Nevis",
                 "St Lucia","St Vincent","St. Lucia","Sudan","Suriname","Swaziland","Sweden","Switzerland","Syria",
                 "Taiwan","Tajikistan","Tanzania","Thailand","Timor L'Este","Togo","Tonga","Trinidad & Tobago","Tunisia","Turkey",
                 "Turkmenistan","Turks & Caicos",
                 "Uganda","Ukraine","United Arab Emirates","United Kingdom","UK","United States","USA","Uruguay","Uzbekistan",
                 "Venezuela","Vietnam","US Virgin Islands",
                 "Wales",
                 "Yemen",
                 "Zambia","Zimbabwe"};

#let's specify the function that select the most similar word
def get_most_similar(word,wordlist):
  top_similarity = 0.25 # set to 0.25 to ensure some similarity
  most_similar_word = word
  for candidate in wordlist:
    similarity = SequenceMatcher(None,word.lower(),candidate.lower()).ratio()
    if similarity > top_similarity:
      top_similarity = similarity
      most_similar_word = candidate

  return most_similar_word

#now apply this function over 'country' column in dataframe
df['country'] = df['country'].apply(lambda x: get_most_similar(x, country_names))

After checking the unique values in the 'country' column, there are some values that can be grouped up. There are also a few instances of the wrong information being entered. These entries need to be removed.

In [22]:
unique_values = df['country'].unique().tolist()
unique_values

['United States',
 'Panama',
 'United Kingdom',
 'Ireland',
 'France',
 'Austria',
 'Germany',
 'Sweden',
 'South Africa',
 'Australia',
 'Montserrat',
 'Belgium',
 'Spain',
 'United States, I physically work in the US but my programs are focused on foreign youth abroad',
 'Netherlands',
 'Mexico',
 'Isle of Man',
 'Japan',
 'Egypt',
 'Finland',
 'China',
 'Malta',
 'Croatia',
 'Armenia',
 'Luxembourg',
 'Latvia',
 'French Polynesia',
 'New Zealand',
 'Italy',
 'Portugal',
 'Saudi Arabia',
 'Brazil',
 'Fully remote - currently employed by US company living in UK',
 'Maldives',
 'Hungary',
 'New Caledonia',
 'Switzerland',
 'Cayman Islands',
 'Gambia',
 'Mongolia',
 'Hong Kong',
 'Kenya',
 'United Arab Emirates',
 'Denmark',
 'India',
 'Burkina Faso',
 'Israel',
 'Slovenia',
 'Poland',
 'Tanzania',
 'Singapore',
 'USA',
 'Bulgaria',
 'Netherlands Antilles',
 'Greece',
 'Norway']

In [24]:
country_mapping = {
    'United States': 'USA',
    'United Kingdom': 'UK'
}

# Use the `replace` method to replace values in the 'Country' column
df['country'] = df['country'].replace(country_mapping, regex=False)

# Removing entries that have irrelevant information in the 'Country' column
df = df[df['country'].str.len() <= 30]

Entries have been removed and so the dataframe's index should be reset.

In [26]:
df = df.reset_index(drop=True)

Certain columns are dropped as they are not needed for our analysis. These columns are not essential for achieving the specified objectives.

In [35]:
df.drop('age', axis=1, inplace=True)
df.drop('job_title_context', axis=1, inplace=True)
df.drop('income_context', axis=1, inplace=True)
df.drop('us_state', axis=1, inplace=True)
df.drop('experience_overall', axis=1, inplace=True) # unnecessary as 'Field_Experience' is more relevant

I've then exported the clean dataframe to a .csv file.

In [37]:
df.to_csv('clean_dataset.csv', index=False)